In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import ast

# ----------------------
# Data Loading & Cleaning
# ----------------------
bills = pd.read_csv("bills.csv")
products = pd.read_csv("products.csv")

# Merge product details into bills
bills = bills.merge(products, on="Product_IDs",how="left")

# Convert Date column to datetime   
bills["Date_Time"] = pd.to_datetime(bills["Date_Time"], errors="coerce")

# Clean Quantity column (handle strings like "[8, 10, 8]")
def clean_quantity(val):
    if isinstance(val, str) and val.startswith("["):
        try:
            return sum(ast.literal_eval(val))
        except:
            return 0.0
    try:
        return float(val)
    except:
        return 0.0

bills["Quantity"] = bills["Quantity"].apply(clean_quantity)

# ----------------------
# Build complete weekly time series
# ----------------------
all_weeks = pd.date_range(bills["Date_Time"].min(), bills["Date_Time"].max(), freq="W-MON")

weekly_full = []
for pid in bills["Product_ID"].unique():
    product_weekly = (
        bills[bills["Product_ID"] == pid]
        .groupby(pd.Grouper(key="Date_Time", freq="W-MON"))["Quantity"]
        .sum()
        .reindex(all_weeks, fill_value=0)
        .reset_index()
        .rename(columns={"index": "Date_Time"})
    )
    product_weekly["Product_ID"] = pid
    weekly_full.append(product_weekly)

weekly = pd.concat(weekly_full, ignore_index=True)

# Monthly aggregation
monthly = bills.groupby(["Product_ID", pd.Grouper(key="Date_Time", freq="M")])["Quantity"].sum().reset_index()

# ----------------------
# Scaling per product
# ----------------------
scalers = {}
weekly["Quantity_scaled"] = 0.0

for pid in weekly["Product_ID"].unique():
    mask = weekly["Product_ID"] == pid
    scaler = MinMaxScaler()
    values = weekly.loc[mask, ["Quantity"]].values.astype(float)
    if values.max() > values.min():  # normal scaling
        weekly.loc[mask, "Quantity_scaled"] = scaler.fit_transform(values)
    else:  # constant series
        weekly.loc[mask, "Quantity_scaled"] = 0.0
        scaler.fit(np.array([[0.0], [1.0]]))
    scalers[pid] = scaler

# ----------------------
# Dataset for PyTorch
# ----------------------
class SalesDataset(Dataset):
    def __init__(self, df, product_id, seq_len=6):
        product_df = df[df["Product_ID"] == product_id].copy()
        product_df = product_df.set_index("Date_Time").asfreq("W-MON", fill_value=0)
        self.data = product_df["Quantity_scaled"].values.astype(float)
        self.seq_len = seq_len

    def __len__(self):
        return max(1, len(self.data) - self.seq_len)

    def __getitem__(self, idx):
        if idx + self.seq_len >= len(self.data):
            idx = len(self.data) - self.seq_len - 1
        x = self.data[idx:idx+self.seq_len]
        y = self.data[idx+self.seq_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# ----------------------
# Model Definition (GRU + ReLU)
# ----------------------
class SalesGRU(nn.Module):
    def __init__(self, hidden_size=64, num_layers=1, dropout=0.2):
        super().__init__()
        self.gru = nn.GRU(input_size=1, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return self.relu(out)

# ----------------------
# Train Model
# ----------------------
def train_model(product_id, epochs=50, seq_len=6):
    dataset = SalesDataset(weekly, product_id, seq_len)
    if len(dataset) < 2:  # too little data
        return None, 0, 0, 0

    split = max(1, int(len(dataset) * 0.8))
    train_data, test_data = torch.utils.data.random_split(dataset, [split, len(dataset) - split])

    train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

    model = SalesGRU()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

    for epoch in range(epochs):
        model.train()
        for X, y in train_loader:
            X = X.unsqueeze(-1)
            y = y.unsqueeze(-1)

            optimizer.zero_grad()
            preds = model(X)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for X, y in test_loader:
            X = X.unsqueeze(-1)
            preds = model(X).squeeze()
            all_preds.extend(preds.numpy())
            all_true.extend(y.numpy())

    scaler = scalers[product_id]
    all_preds = scaler.inverse_transform(np.array(all_preds).reshape(-1,1)).flatten()
    all_true = scaler.inverse_transform(np.array(all_true).reshape(-1,1)).flatten()

    if len(all_true) > 0:
        mae = mean_absolute_error(all_true, all_preds)
        rmse = mean_squared_error(all_true, all_preds, squared=False)
        mape = np.mean(np.abs((all_true - all_preds) / np.maximum(all_true, 1e-5))) * 100
        acc = max(0, 100 - mape)
    else:
        mae, rmse, acc = 0, 0, 0

    return model, mae, rmse, acc

# ----------------------
# Predict Future
# ----------------------
def predict_future(model, product_id, seq_len=6, weeks=8):
    dataset = SalesDataset(weekly, product_id, seq_len)
    seq = dataset.data[-seq_len:]
    preds = []

    for _ in range(weeks):
        x = torch.tensor(seq, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
        with torch.no_grad():
            pred = model(x).item()
        preds.append(pred)
        seq = np.append(seq[1:], pred)

    scaler_local = scalers[product_id]
    preds_inverse = scaler_local.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()

    future_dates = pd.date_range(start=weekly["Date_Time"].max() + pd.Timedelta(weeks=1), periods=weeks, freq="W-MON")
    return pd.DataFrame({"Week": future_dates, "Predicted": preds_inverse})

# ----------------------
# Extra Analysis
# ----------------------
def average_sales():
    weekly_avg = weekly.groupby("Product_ID")["Quantity"].mean().reset_index(name="Avg_Weekly_Sales")
    monthly_avg = monthly.groupby("Product_ID")["Quantity"].mean().reset_index(name="Avg_Monthly_Sales")
    return weekly_avg, monthly_avg

def brand_distribution():
    return bills.groupby(["Brand", "Product_ID"])["Quantity"].sum().reset_index(name="Total_Sales")

# ----------------------
# Run for all products & Save Excel
# ----------------------
all_future_preds = {}
accuracy_records = []

for pid in weekly["Product_ID"].unique():
    print(f"\nTraining model for {pid}...")
    model, mae, rmse, acc = train_model(pid, epochs=50, seq_len=6)

    if model is not None:
        future_preds = predict_future(model, pid, weeks=12)
        all_future_preds[pid] = future_preds

    accuracy_records.append({"Product_ID": pid, "MAE": mae, "RMSE": rmse, "Accuracy(%)": acc})

weekly_avg, monthly_avg = average_sales()
brand_sales = brand_distribution()
accuracy_df = pd.DataFrame(accuracy_records)

with pd.ExcelWriter("sales_outputs.xlsx") as writer:
    for pid, preds in all_future_preds.items():
        preds.to_excel(writer, sheet_name=f"Future_{pid[:10]}", index=False)
    weekly_avg.to_excel(writer, sheet_name="Weekly_Avg", index=False)
    monthly_avg.to_excel(writer, sheet_name="Monthly_Avg", index=False)
    brand_sales.to_excel(writer, sheet_name="Brand_Sales", index=False)
    accuracy_df.to_excel(writer, sheet_name="Accuracy", index=False)

print("\nAll outputs saved to sales_outputs.xlsx")

KeyError: 'Product_IDs'